# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 3.0.36 which is incompatible.
jupyter-console 6.4.3 has requirement jupyter-client>=7.0.0, but you'll have jupyter-client 5.2.4 which is incompatible.


The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [2]:
from unityagents import UnityEnvironment

import torch
import numpy as np
from ddpg_agent import Agent
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline


# select this option to load version 1 (with a single agent) of the environment
# env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64')




# select this option to load version 2 (with 20 agents) of the environment
env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

agent = Agent(state_size=state_size, action_size=action_size, random_seed=10)

import random

gen = [random.random() for i in range(10)]

print(gen)

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.75471878e+00  -1.00000000e+00
   5.55726624e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
  -1.68164849e-01]
[0.5714025946899135, 0.4288890546751146, 0.5780913011344704, 0.20609823213950174, 0.81332125135732, 0.8235888725334455, 0.6534725339011758, 0.16022955651881965, 0.5206693596399246, 0.32777281162209315]


In [9]:
def ddpg(n_episodes=1000, max_t=300):
    
    scores_deque = deque(maxlen=100)
    scores = []
    max_score = -np.Inf
    
    for i_episode in range(1, n_episodes+1):
        
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations
        
        #se resetea el noise aquí!
        agent.reset()
        
        agent_scores = [0 for i in range(len(env_info.agents))]
        
        for t in range(max_t):
            
            print(states.shape)
            
            actions = agent.act(states)
            
            print(actions.shape)
            
            env_info = env.step(actions)[brain_name]
            
            #print("env_info shape: ", env_info.shape)
            
            rewards = env_info.rewards
            next_states = env_info.vector_observations
            dones = env_info.local_done
            
            print(len(rewards))
            
            agent.step(states, actions, rewards, next_states, dones)
            
            states = next_states
            agent_scores += rewards
            
            if dones:
                break 
        
        scores_mean = np.mean(agent_scores)
        scores_deque.append(scores_mean)
        
        #print(scores_deque)
        
        scores.append(scores_mean)
        
        #print(scores)
        print('\rEpisode {}\tAverage Score: {:.2f}\tScore: {:.2f}'.format(i_episode, np.mean(scores_deque), scores_mean), end="")
        if i_episode % 100 == 0:
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))   
    return scores

scores = ddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 1	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 2	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 3	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 4	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 5	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 6	Avera

Episode 48	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 49	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 50	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 51	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 52	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 53	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targ

Episode 96	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 97	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 98	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 99	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 100	Average Score: 0.00	Score: 0.00
(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 101	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_t

Episode 144	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 145	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 146	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 147	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 148	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 149	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])


Episode 192	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 193	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 194	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 195	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 196	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 197	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])


Episode 240	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 241	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 242	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 243	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 244	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 245	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])


Episode 288	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 289	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 290	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 291	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 292	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 293	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])


Episode 336	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 337	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 338	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 339	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 340	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 341	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])


Episode 384	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 385	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 386	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 387	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 388	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])
Q_targets_next:  torch.Size([128, 1])
Episode 389	Average Score: 0.00	Score: 0.00(20, 33)
(20, 4)
20
next_states:  torch.Size([128, 33])
actions_next:  torch.Size([128, 4])


KeyboardInterrupt: 

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [6]:
env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)

scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    # lo que sea que hagas  aqui va!
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.07599999830126762


When finished, you can close the environment.

In [7]:
#env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 